### 作業
請使用不同的資料集，並使用 hyper-parameter search 的方式，看能不能找出最佳的超參數組合

In [24]:
from sklearn import datasets, metrics, linear_model
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd

In [3]:
digits = datasets.load_digits()
print(f'digits.data.shape: {digits.data.shape}')
print(f'digits.data.dtype: {digits.data.dtype}')
print(f'digits.keys: {digits.keys()}')
print(f'digits.target_names: {digits.target_names}')

digits.data.shape: (1797, 64)
digits.data.dtype: float64
digits.keys: dict_keys(['data', 'target', 'frame', 'feature_names', 'target_names', 'images', 'DESCR'])
digits.target_names: [0 1 2 3 4 5 6 7 8 9]


In [5]:
print(digits.DESCR)

.. _digits_dataset:

Optical recognition of handwritten digits dataset
--------------------------------------------------

**Data Set Characteristics:**

    :Number of Instances: 1797
    :Number of Attributes: 64
    :Attribute Information: 8x8 image of integer pixels in the range 0..16.
    :Missing Attribute Values: None
    :Creator: E. Alpaydin (alpaydin '@' boun.edu.tr)
    :Date: July; 1998

This is a copy of the test set of the UCI ML hand-written digits datasets
https://archive.ics.uci.edu/ml/datasets/Optical+Recognition+of+Handwritten+Digits

The data set contains images of hand-written digits: 10 classes where
each class refers to a digit.

Preprocessing programs made available by NIST were used to extract
normalized bitmaps of handwritten digits from a preprinted form. From a
total of 43 people, 30 contributed to the training set and different 13
to the test set. 32x32 bitmaps are divided into nonoverlapping blocks of
4x4 and the number of on pixels are counted in each blo

In [9]:
data = pd.DataFrame(digits.data, columns=digits.feature_names)
data['target'] = digits.target

data.head()

,pixel_0_0,pixel_0_1,pixel_0_2,pixel_0_3,pixel_0_4,pixel_0_5,pixel_0_6,pixel_0_7,pixel_1_0,pixel_1_1,...,pixel_6_7,pixel_7_0,pixel_7_1,pixel_7_2,pixel_7_3,pixel_7_4,pixel_7_5,pixel_7_6,pixel_7_7,target
0,0.0,0.0,5.0,13.0,9.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,6.0,13.0,10.0,0.0,0.0,0.0,0
1,0.0,0.0,0.0,12.0,13.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,11.0,16.0,10.0,0.0,0.0,1
2,0.0,0.0,0.0,4.0,15.0,12.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,11.0,16.0,9.0,0.0,2
3,0.0,0.0,7.0,15.0,13.0,1.0,0.0,0.0,0.0,8.0,...,0.0,0.0,0.0,7.0,13.0,13.0,9.0,0.0,0.0,3
4,0.0,0.0,0.0,1.0,11.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,16.0,4.0,0.0,0.0,4


In [12]:
x, y = data.iloc[:, :-1], data['target']
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.1, random_state=7)

In [17]:
# 建立模型
gbcl = GradientBoostingClassifier()

gbcl.fit(train_x, train_y)
pred_test_y = gbcl.predict(test_x)
acc = metrics.accuracy_score(test_y, pred_test_y)
print("Acuuracy: ", acc)

Acuuracy:  0.95


In [21]:
n_estimators = [10, 25, 50, 75, 100, 200]
max_depth = [2, 4, 6, 8, 10]
grid_param = dict(n_estimators=n_estimators, max_depth=max_depth)
grid_param

{'n_estimators': [10, 25, 50, 75, 100, 200], 'max_depth': [2, 4, 6, 8, 10]}

In [26]:
## 建立搜尋物件，放入模型及參數組合字典 (n_jobs=-1 會使用全部 cpu 平行運算)
## GridSearchCV:https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html
## scoring選擇 https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
grid_search = GridSearchCV(gbcl, grid_param, scoring="accuracy", n_jobs=-1, verbose=1)
grid_result = grid_search.fit(train_x, train_y)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


In [27]:
# 印出最佳結果與最佳參數
print("Best Accuracy: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best Accuracy: 0.973396 using {'max_depth': 2, 'n_estimators': 200}


In [28]:
grid_result.best_params_

{'max_depth': 2, 'n_estimators': 200}

In [29]:
# 建立模型
gbcl_bestparam = GradientBoostingClassifier(max_depth=grid_result.best_params_['max_depth'], \
                                  n_estimators=grid_result.best_params_['n_estimators'])

gbcl_bestparam.fit(train_x, train_y)
pred_bestparam_test_y = gbcl_bestparam.predict(test_x)
acc = metrics.accuracy_score(test_y, pred_bestparam_test_y)
print("Acuuracy: ", acc)

Acuuracy:  0.9555555555555556
